# Biomedical Data Analysis - Lab 4

Group members:

- Enrico Maria Marinelli
- Francesco Pio Capoccello
- Juras LukaŠevičius

### Libraries used in practical session:

In [3]:
import pandas as pd

## Block I. Data loading and basic description

We load the dataset using the pandas’ load_csv function, remember to specify semicolon (;) as
the separator

In [4]:
df=pd.read_csv("inadvance_synth.csv",sep=";")

In [5]:
df.head()

,Unnamed: 0,age,barthel,charlson,codidiagingreso,codservicioreal,creatinina,drg,estancias,glucosa,...,metastatic_tumor,num_grupoact3_HOSP,numurgenciasprevias,potasio,proteina_c_reactiva,rdw_cv,rdw_sd,sodio,urea,label
0,0,80,NaN,4.0,NaN,HNEM,0.42,141.0,7.0,110.0,...,0.0,NaN,6.0,4.653334,NaN,24.9,83.1,145.0,NaN,0.0
1,1,67,NaN,5.0,NaN,HNEF,NaN,444.0,11.0,NaN,...,0.0,NaN,1.0,NaN,NaN,14.8,46.3,NaN,NaN,0.0
2,2,75,NaN,13.0,M97303,HHEM,0.92,691.0,14.0,NaN,...,0.0,NaN,16.0,5.153334,1.090658,23.1,57.2,135.0,NaN,0.0
3,3,75,NaN,0.0,560.81,HCDG,0.96,NaN,3.0,87.3,...,0.0,NaN,4.0,3.323334,NaN,13.0,42.4,138.0,23.052822,0.0
4,4,81,95.0,6.0,NaN,HCAR,1.54,194.0,4.0,95.7,...,0.0,6.0,3.0,3.783334,NaN,14.9,51.7,140.0,60.752822,0.0


### 1. What is the size of the dataframe?

In [9]:
df.shape

(38416, 22)

### 2. What is the mean age?

In [11]:
mean = df.age.mean()
print(mean)

79.4033215326947


### 3. What is the age standard deviation (std)?

In [12]:
std=df.age.std()
print(std)

8.361737254116035


### 4. Which is the variable with the most amount of missing values? Can you list the name of the variables, sorting them by number of missing values?

In [13]:
valorinulli = df.isnull().sum()
features_ordinate = valorinulli.sort_values(ascending=False)
print(features_ordinate)

barthel                28198
num_grupoact3_HOSP     21001
proteina_c_reactiva    18175
urea                   14260
codidiagingreso        11378
glucosa                 9228
potasio                 8646
rdw_sd                  8334
rdw_cv                  8334
leucocitos              8302
hematocrito             8302
sodio                   8142
creatinina              7951
drg                     3640
numurgenciasprevias      552
charlson                  77
metastatic_tumor          77
age                        0
label                      0
codservicioreal            0
estancias                  0
Unnamed: 0                 0
dtype: int64


### 5. Name which are the categorical variables.

In [14]:
variabili_categoriche = df.select_dtypes(include=['object'])
print(variabili_categoriche.columns.tolist())

['codidiagingreso', 'codservicioreal']


Two additional categorical variables to the existing `codidiagingreso` and `codservicioreal` are `metastatic_tumor` and `label`, since their only values are $1$ and $2$.

### 6. Extract the ‘label’ column to another variable. How many positive cases there are? And negatives?


In [15]:
df['label'].value_counts(ascending=True)

1.0    13431
0.0    24985
Name: label, dtype: int64

## Block II. Data preprocessing

Split the dataset in two: train (80%) and test (20%). Use a seed to allow replication.

Implement a method to deal with categorical variables.

Implement a method to deal with missing values using simple techniques such as the imputation by the mean or the median.

The transformation applied on the training set of the data should be the same applied to the test set